### 1. Import libraries yang dibutuhkan

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

### 2. Pembuatan model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

C:\Users\LENOVO\anaconda3\envs\porto\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### 3. Hitung dan encode sentimen

In [16]:
token = tokenizer.encode('This is the greatest moment of my life', return_tensors='pt')

In [17]:
result = model(token)

In [18]:
result.logits

tensor([[-0.9327, -1.6672, -1.2393, -0.0404,  3.1583]],
       grad_fn=<AddmmBackward0>)

In [19]:
int(torch.argmax(result.logits))+1

5

### 4. Ambil ulasan dari Yelp

In [20]:
r = requests.get('https://www.yelp.com/biz/wingstop-san-francisco-5')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [24]:
results[0].text

'The Food is great with many different flavors with different spice levels, different kinds of fries and there homemade ranch is amazing!! highly recommen!'

### 5. Memasukkan ulasan ke dalam DataFrame

In [25]:
import pandas as pd
import numpy as np

In [26]:
data = pd.DataFrame(np.array(reviews), columns=['review'])

In [27]:
data.head()

,review
0,The Food is great with many different flavors ...
1,The best alitas! The Service It's weird that I...
2,"If you look at all the reviews online, you wou..."
3,Do not order from this location. There's a wom...
4,They charged me $40 for 10 boneless wings... I...


In [28]:
data.tail()

,review
5,Idk what that place is but it is not a wing st...
6,Wingstop Downtown!?!? In the SoMa? Next to/beh...
7,My ratings are never this low. I was on their...
8,Awful service I placed wings for delivery and ...
9,when you order online the food is not ready on...


In [30]:
data['review'].iloc[0]

'The Food is great with many different flavors with different spice levels, different kinds of fries and there homemade ranch is amazing!! highly recommen!'

### 6. Berikan skor sentimen pada ulasan menggunakan model BERT

In [31]:
def skor_sentimen(review):
    token = tokenizer.encode(review, return_tensors='pt')
    result = model(token)
    return int(torch.argmax(result.logits))+1

In [32]:
skor_sentimen(data['review'].iloc[0])

5

In [33]:
data['sentiment'] = data['review'].apply(lambda x: skor_sentimen(x[:512]))

In [34]:
data

,review,sentiment
0,The Food is great with many different flavors ...,5
1,The best alitas! The Service It's weird that I...,5
2,"If you look at all the reviews online, you wou...",3
3,Do not order from this location. There's a wom...,1
4,They charged me $40 for 10 boneless wings... I...,1
5,Idk what that place is but it is not a wing st...,3
6,Wingstop Downtown!?!? In the SoMa? Next to/beh...,1
7,My ratings are never this low. I was on their...,1
8,Awful service I placed wings for delivery and ...,1
9,when you order online the food is not ready on...,1


### 7. Simpan hasil analisis sentimen

In [35]:
data.to_csv('hasil_analisis_sentimen_yelp.csv', encoding='utf-8', index=False)